# **Training Notebook (NovaSBE X GregoryAI)**

![Description of the image](../images/train_tune_pipeline_diagram.png)

## 1. Import libraries

## 2. Download articles

## 3. Clean and Preprocess

## 4. Data Split into train, validation and test sets

## 5. Pseudo Labelling

## 5. Train the Model and Store Model Weights

## 6. Show Model Performance Results